In [1]:
import pandas as pd
import numpy as np

# KNN
from sklearn import neighbors

# Accuracy check
from sklearn.metrics import accuracy_score

# For deskewing
from scipy.ndimage import interpolation


In [2]:
# Init
n_neighbors = 5


In [3]:
# Deskew help funcion (Not written by me) - https://fsix.github.io/mnist/Deskewing.html

In [4]:
def moments(image):
    c0,c1 = np.mgrid[:image.shape[0],:image.shape[1]] # A trick in numPy to create a mesh grid
    totalImage = np.sum(image) #sum of pixels
    m0 = np.sum(c0*image)/totalImage #mu_x
    m1 = np.sum(c1*image)/totalImage #mu_y
    m00 = np.sum((c0-m0)**2*image)/totalImage #var(x)
    m11 = np.sum((c1-m1)**2*image)/totalImage #var(y)
    m01 = np.sum((c0-m0)*(c1-m1)*image)/totalImage #covariance(x,y)
    mu_vector = np.array([m0,m1]) # Notice that these are \mu_x, \mu_y respectively
    covariance_matrix = np.array([[m00,m01],[m01,m11]]) # Do you see a similarity between the covariance matrix
    return mu_vector, covariance_matrix

In [12]:
def deskew(image):
    c,v = moments(image)
    alpha = v[0,1]/v[0,0]
    affine = np.array([[1,0],[alpha,1]])
    ocenter = np.array(image.shape)/2.0
    offset = c-np.dot(affine,ocenter)
    return interpolation.affine_transform(image,affine,offset=offset)

In [ ]:
# Get the data

In [6]:
train_data = pd.read_csv('mnist_train.csv',header=None)
test_data = pd.read_csv('mnist_test.csv',header=None)

In [7]:
train_y = train_data.iloc[:,0].values
test_y = test_data.iloc[:,0].values

In [8]:
train_X = train_data.iloc[:,1:].values
test_X = test_data.iloc[:,1:].values

# KNN

In [9]:
# Let's try KNN
clf = neighbors.KNeighborsClassifier(n_neighbors)

In [17]:
# Fit the model
clf.fit(train_X, train_y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [18]:
predictions = clf.predict(test_X)

In [19]:
print("Testing Error = " +str(1-accuracy_score(predictions,test_y)))


Testing Error = 0.0312


# KNN Features

In [13]:
# Deskew all samples
def deskew_all(data):
    currents = []
    for i in range(len(data)):
        currents.append(deskew(data[i].reshape(28,28)).flatten())
    return np.array(currents)

train_X_deskewed = deskew_all(train_X)
test_X_deskewed = deskew_all(test_X)

In [14]:
# Fit the model
clf.fit(train_X_deskewed, train_y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [15]:
predictions = clf.predict(test_X_deskewed)

In [16]:
print("Testing Error = " +str(1-accuracy_score(predictions,test_y)))


Testing Error = 0.0173
